# Homework V

## Task 1

Demographic parity:
$\frac{P(\hat{Y} | A=Blue)}{P(\hat{Y} | A=Red)} = \frac{0.65}{0.5} = 1.3$
Equal opportunity:
$\frac{P(\hat{Y} | A=Blue, Y=1)}{P(\hat{Y} | A=Red, Y=1)} = \frac{\frac{3}{4}}{0.5} = 1.5$
Predictive rate parity:
$\frac{P(Y=1 | A=Blue, \hat{Y}=1)}{P(Y=1 | A=Red, \hat{Y}=1)} = \frac{\frac{60}{65}}{0.5} \approx 1.85$

## Task 2

## 2

Below are fairness statistics for XGBClassifier model trained on adult.csv. The protected variable is gender. As we can see two statistics are out of bounds (predictive equality ratio and statistical parity ratio).

![alt text](img2.png "img2")

## 3

Below is the comparision of fairness statistics for XGBClassifier and RandomForestClassifier. For the second model statistics are worse and for this also equal opportunity ratio is out of bounds.

![alt text](img3.png "img3")

## 4

Below is the comparision of fairness statistics for XGBClassifier and XGBClassifier with applied bias mitigation technique. For the second model statistics are worse and for this also equal opportunity ratio is out of bounds. In this case this technique didn't help, because now even more statistics is out of bounds. The most interested in my opinion is equal opportunity ratio, because earlier was a little below 1 and now is more than 1.2.

![alt text](img4.png "img4")

## 5

Below is the comparison of all 3 models and their performances. There isn't highly visible correlation between performanc and explanation. ONe thing which is worth to noticed is that reweighted model is slightly worse than original.

![alt text](img51.png "img51")
![alt text](img52.png "img52")

## Appendix

In [70]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

import dalex

from copy import copy

In [37]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [38]:
data.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


## One-hot encoding of categorical columns

We are using one-hot encoding for categorical data

In [39]:
data = pd.get_dummies(data, columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income'],
                      drop_first=True)
data.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income_>50K
0,25,226802,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,38,89814,9,0,0,50,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,28,336951,12,0,0,40,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
3,44,160323,10,7688,0,40,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
4,18,103497,10,0,0,30,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Splitting data into training and test set

In [40]:
x, y = data.drop(columns='income_>50K'), data['income_>50K']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## Training models

In [57]:
model = XGBClassifier(n_estimators=100, max_depth=4).fit(x_train, y_train)
explainer = dalex.Explainer(model, x_test, y_test)

Preparation of a new explainer is initiated

  -> data              : 9769 rows 100 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9769 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7f9e513ac700> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 1.32e-05, mean = 0.237, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.977, mean = 0.00316, max = 1.0
  -> model_info        : package xgboost

A new explainer has been created!


In [42]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.641759,0.777146,0.702993,0.869997,0.926632


In [43]:
protected_variable = x_test['gender_Male'].apply(lambda x: 'male' if x else 'female')
privileged_group = 'male'
fobject = explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR      ACC       PPV       FPR       STP
female  0.883614  1.12201  0.989717  0.243902  0.302326


In [44]:
fobject.plot()

In [45]:
model2 = RandomForestClassifier(n_estimators=100, max_depth=4).fit(x_train, y_train)
explainer2 = dalex.Explainer(model2, x_test, y_test)

Preparation of a new explainer is initiated

  -> data              : 9769 rows 100 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9769 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7f9e513ac700> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.


/home/maciej/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



  -> predicted values  : min = 0.0466, mean = 0.239, max = 0.712
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.604, mean = 0.00106, max = 0.942
  -> model_info        : package sklearn

A new explainer has been created!


In [46]:
explainer2.model_performance()

,recall,precision,f1,accuracy,auc
RandomForestClassifier,0.265158,0.928251,0.412488,0.818917,0.897834


In [47]:
protected_variable = x_test['gender_Male'].apply(lambda x: 'male' if x else 'female')
privileged_group = 'male'
fobject2 = explainer2.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject2.fairness_check()

Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV  FPR       STP
female  0.568905  1.179819  1.042162  0.1  0.180851


In [48]:
fobject2.plot(fobject)

In [58]:
protected_variable_train = x_train['gender_Male'].apply(lambda x: "male" if x else "female")

sample_weight = dalex.fairness.reweight(
    protected_variable_train,
    y_train,
    verbose=False
)
model_reweight = copy(model)
model_reweight.fit(x_train, y_train, sample_weight=sample_weight)
explainer_reweight = dalex.Explainer(
    model_reweight,
    x_test,
    y_test,
    label='XGBClassifier with Reweight mitigation',
    verbose=False
)
fobject_reweight = explainer_reweight.model_fairness(
    protected_variable,
    privileged_group
)

In [ ]:
fobject_reweight.plot(fobject)

In [ ]:
fobject_reweight.plot([fobject, fobject2])

In [68]:
mp1 = explainer.model_performance().result
mp2 = explainer2.model_performance().result
mp3 = explainer_reweight.model_performance().result

In [69]:
pd.concat([mp1, mp2, mp3])

,recall,precision,f1,accuracy,auc
XGBClassifier,0.641759,0.777146,0.702993,0.869997,0.926632
RandomForestClassifier,0.265158,0.928251,0.412488,0.818917,0.897834
XGBClassifier with Reweight mitigation,0.598634,0.778024,0.676641,0.862831,0.920149
